In [1]:
import pandas as pd
import numpy as np
import df_build as build

In [2]:
team = 'Man City'

In [3]:
team_focus = build.df_team_focus(build.csv_to_df(), team)

/Users/zschulzb/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## Modeling

In [4]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix

In [5]:
bal_rf = BalancedRandomForestClassifier()

In [6]:
model_data = team_focus.copy()

In [8]:
game_to_predict = '2019-10-02'

In [9]:
model_data = team_focus.copy()

In [10]:
model_data.head()

,home_advan,team_shots,team_target_shots,team_goals,team_fouls,team_corners,team_yellows,team_reds,opponent,oppon_shots,oppon_target_shots,oppon_goals,oppon_fouls,oppon_corners,oppon_yellows,oppon_reds,season_num
Date,,,,,,,,,,,,,,,,,
2019-12-05,False,20,9,4,8,6,0,0,Brighton,6,2,1,12,2,0,0,10
2019-10-02,True,15,9,6,9,2,1,0,Chelsea,12,4,0,13,2,2,0,10
2019-09-03,True,19,9,3,11,9,1,0,Watford,2,1,1,7,1,1,0,10
2019-06-05,True,19,5,1,12,11,3,0,Leicester,7,2,0,5,0,2,0,10
2019-06-02,False,15,4,2,6,6,1,0,Everton,4,1,0,12,5,1,0,10


In [11]:
y = model_data.pop('team_reds')
y = y[y.index < game_to_predict]
y.shift();

Date
2019-09-03    NaN
2019-06-05    0.0
2019-06-02    0.0
2019-04-28    0.0
2019-04-24    0.0
             ... 
2009-09-20    0.0
2009-09-12    0.0
2009-08-30    0.0
2009-08-22    0.0
2009-08-15    0.0
Name: team_reds, Length: 378, dtype: float64

In [25]:
y_test = y[:100]
y_train = y[100:]

In [13]:
X = model_data.drop(['opponent', 'season_num'], axis=1)
home_flag = X.pop('home_advan')

In [14]:
X = X.sort_index().rolling(3).mean()
X = X.sort_index()[::-1]

In [16]:
X.insert(0, 'home_advan', home_flag)

In [18]:
X_train = X[100:-2]
X_test = X[:100]

In [28]:
bal_rf.fit(X_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [278, 277]

In [60]:
bal_rf.score(X_test, y_test)

0.82

In [62]:
y_predict = bal_rf.predict(X_test)

In [65]:
confusion_matrix(y_test, y_predict)

array([[82,  2, 12],
       [ 2,  0,  2],
       [ 0,  0,  0]])

In [67]:
y_train.unique()

array([0, 1, 2])